In [71]:
using DataFrames, DataFramesMeta, Optim, Revise, Serialization, DebuggingUtilities, LinearAlgebra, Statistics
using JuMP, Ipopt

In [11]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

In [12]:
test1state = true
if test1state 
    dirpath = "/export/storage_adgandhi/MiscLi/factract/analysis"
else
    dirpath = "jls"
end;
ec = deserialize("$dirpath/ec.jls");

---
### jump

In [69]:
# create some indices 
# t_indexer[jj] is the vector of markets (i.e. their indices) served by facility jj
t_indexer = [[] for _ in 1:length(ec.firms)]
for tt in eachindex(ec.tracts)
    for ind in ec.tracts[tt].inds
        push!(t_indexer[ind], tt)
    end
end

# positionin[jj,tt] is firm jj's position in tract tt's vectors (e.g. the vector of market shares in tt)
positionin = Matrix(undef, n_firms, n_tracts)
for jj in 1:n_firms
    for tt in t_indexer[jj]
        positionin[jj,tt] = findall(x->x == jj, j_indexer[tt])[1]
    end
end

In [131]:
n_firms = length(ec.firms);
n_firms_intract = [t.n_firms for t in ec.tracts]
n_tracts = length(ec.tracts);
nX = 2
nD = 2
X = [t.X[:,1:nX] for t in ec.tracts]
D = [t.D[:,1:nD] for t in ec.tracts]
j_indexer = [t.inds for t in ec.tracts]
M = [t.M for t in ec.tracts]
q_obs = ec.q_obs;

In [132]:
model = Model(Ipopt.Optimizer)
@variables(model, begin
    θx[1:nX]
    θd[1:nD]
end)

u = @NLexpression(model,
    [tt in 1:n_tracts, jj in j_indexer[tt]],
    sum(θx[ix] * X[tt][positionin[jj,tt],ix] for ix in 1:nX) + sum(θd[id] * D[tt][positionin[jj,tt],id] for id in 1:nD))

expu = @NLexpression(model,
    [tt in 1:n_tracts, jj in j_indexer[tt]],
    exp(u[tt,jj]))

denom = @NLexpression(model,
    [tt in 1:n_tracts],
    1+ sum(expu[tt,jj] for jj in j_indexer[tt]))

share = @NLexpression(model,
    [tt in 1:n_tracts, jj in j_indexer[tt]],
    expu[tt,jj] / denom[tt])

mktq = @NLexpression(model,
    [tt in 1:n_tracts, jj in j_indexer[tt]],
    M[tt] * share[tt,jj])

firmq = @NLexpression(model,
    [jj in 1:n_firms],
    sum(mktq[tt, jj] for tt in t_indexer[jj]))

@NLobjective(model, Min, sum((firmq[jj] - q_obs[jj])^2 for jj in 1:n_firms))

optimize!(model)


This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.4612031e+08 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [133]:
value.(θx)

2-element Vector{Float64}:
 -0.31528325398934093
 -0.2601845414584704

In [134]:
value.(θd)

2-element Vector{Float64}:
 -88.0723151986671
 375.3906809161418

In [135]:
# index with q_preds[jj]
q_preds = value.(firmq);

# index with share_preds[tt,jj]
share_preds = value.(share);
mktq_preds = value.(mktq);

In [136]:
# θD1 = value.(θd);
θD1, θD2 = value.(θd);

In [137]:
# quadratic distance
ηDj = [sum([mktq_preds[tt,jj]*(1-share_preds[tt,jj])*(θD2*D[tt][positionin[jj,tt],1] + θD1)*D[tt][positionin[jj,tt],1] for tt in t_indexer[jj]]) / q_preds[jj] for jj in 1:n_firms];

# linear distance
# ηDj_lin = [sum([mktq_preds[tt,jj]*(1-share_preds[tt,jj])*(θD1)*D[tt][positionin[jj,tt],1] for tt in t_indexer[jj]]) / q_preds[jj] for jj in 1:n_firms];

In [138]:
mean(ηDj)

-2.674955301327756

In [ ]:
mean(ηDj_lin)

1-element Vector{Float64}:
 -1.7684851296113835